In [1]:
print("hii")

hii


In [2]:
import os


#define the llm
from crewai import LLM
llm = LLM(
    model = "gemini/gemini-2.0-flash",
    temperature = 0.1,
    api_key=os.environ.get('GOOGLE_API_KEY')
)

In [4]:
from crewai.tools import BaseTool

class ReplaceJargonsTool(BaseTool):

    name: str = "Jargon replacement tool"
    description : str = "Replaces jargon with more specific terms. "

    def _run(self, email: str) -> str:
        replacements = {
            "PRX": "Project Phoenix (internal AI revamp project)",
            "TAS": "technical architecture stack",
            "DBX": "client database cluster",
            "SDS": "Smart Data Syncer",
            "SYNCBOT": "internal standup assistant bot",
            "WIP": "in progress",
            "POC": "proof of concept",
            "ping": "reach out"
        }
        suggestions = []
        email_lower = email.lower()
        for jargon, replacement in replacements.items():
            if jargon.lower() in email_lower:
                suggestions.append(f"Consider replacing '{jargon}' with '{replacement}'")

        return "\n".join(suggestions) if suggestions else "No jargon or internal abbreviations detected."

jt = ReplaceJargonsTool()

In [5]:
original_email = """
looping in Priya. TAS and PRX updates are in the deck. ETA for SDS integration is Friday.
Let's sync up tomorrow if SYNCBOT allows 😄. ping me if any blockers.
"""

jt.run(original_email)

Using Tool: Jargon replacement tool


"Consider replacing 'PRX' with 'Project Phoenix (internal AI revamp project)'\nConsider replacing 'TAS' with 'technical architecture stack'\nConsider replacing 'SDS' with 'Smart Data Syncer'\nConsider replacing 'SYNCBOT' with 'internal standup assistant bot'\nConsider replacing 'ping' with 'reach out'"

In [7]:
from crewai import Agent,Task,Crew

# define an agent
email_assistant = Agent(
    role = "Email Assistant Agent",
    goal = "Improve emails and make them sound clear and professional",
    backstory = "A highly experienced skilled professional in email writing",
    vaerbose = True,
    tools=[jt],
    llm = llm
)

In [8]:
## create task what agent has to perform
email_task = Task(
    description=f""" Take the following rough email and rewrite it into a professional and polished version
    Expand abbreviations.
        {original_email}
      """,
      agent=email_assistant,
      expected_output="a professional written email with proper formatting and content"
)

In [9]:
crew  = Crew(
    agents=[email_assistant],
    tasks=[email_task],
    verbose=True,
)


crew.kickoff()

┌────────────────────────── Crew Execution Started ───────────────────────────┐
│                                                                             │
│  Crew Execution Started                                                     │
│  Name: crew                                                                 │
│  ID: d408800d-db13-463f-b54e-6b137fc08799                                   │
│  Tool Args:                                                                 │
│                                                                             │
│                                                                             │
└─────────────────────────────────────────────────────────────────────────────┘

🚀 Crew: crew
└── 📋 Task: 0776914d-9e5f-4f79-b00e-f8fab5374cbe
    Status: Executing Task...
┌───────────────────────────── 🤖 Agent Started ──────────────────────────────┐
│                                                                             │
│  Agent: Email Assistant Ag

CrewOutput(raw="Hi Team,\n\nI'm including Priya in this email. Updates on the technical architecture stack (TAS) and Project Phoenix (internal AI revamp project) are available in the presentation deck. The estimated time of arrival (ETA) for Smart Data Syncer (SDS) integration is Friday.\n\nLet's schedule a meeting tomorrow, pending the availability of the internal standup assistant bot. Please reach out to me if you encounter any impediments.\n\nThanks,\n[Your Name]", pydantic=None, json_dict=None, tasks_output=[TaskOutput(description=" Take the following rough email and rewrite it into a professional and polished version\n    Expand abbreviations.\n        \nlooping in Priya. TAS and PRX updates are in the deck. ETA for SDS integration is Friday.\nLet's sync up tomorrow if SYNCBOT allows 😄. ping me if any blockers.\n\n      ", name=None, expected_output='a professional written email with proper formatting and content', summary=' Take the following rough email and rewrite it into...',

c:\Users\Star\AppData\Local\Programs\Python\Python311\Lib\site-packages\pydantic\main.py:463: UserWarning: Pydantic serializer warnings:
  PydanticSerializationUnexpectedValue(Expected 9 fields but got 5: Expected `Message` - serialized value may not be as expected [input_value=Message(content='I need t...er_specific_fields=None), input_type=Message])
  PydanticSerializationUnexpectedValue(Expected `StreamingChoices` - serialized value may not be as expected [input_value=Choices(finish_reason='st...r_specific_fields=None)), input_type=Choices])
  return self.__pydantic_serializer__.to_python(
c:\Users\Star\AppData\Local\Programs\Python\Python311\Lib\site-packages\pydantic\main.py:463: UserWarning: Pydantic serializer warnings:
  PydanticSerializationUnexpectedValue(Expected 9 fields but got 5: Expected `Message` - serialized value may not be as expected [input_value=Message(content='\nConsid...er_specific_fields=None), input_type=Message])
  PydanticSerializationUnexpectedValue(Expecte

In [14]:
crewwithoutVerbose  = Crew(
    agents=[email_assistant],
    tasks=[email_task],
    verbose=False,
)


resp = crewwithoutVerbose.kickoff()

In [15]:
resp.raw

'Subject: Updates on Technical Architecture Stack, Project Phoenix, and Smart Data Syncer Integration\n\nHi Team,\n\nI am including Priya in this email to keep her in the loop. The technical architecture stack and Project Phoenix (internal AI revamp project) updates are available in the presentation deck. The estimated time of arrival for the Smart Data Syncer integration is Friday.\n\nI would like to schedule a meeting tomorrow to discuss these updates further, pending the availability of our internal standup assistant bot. Please reach out to me if you foresee any impediments.\n\nThanks,\n\n[Your Name]'